In [1]:
from layers import *
from lasagne.layers import *

from lasagne.nonlinearities import LeakyRectify

from collections import OrderedDict

import theano

In [2]:
def buildGenerator(n_chan=3, filter_size=(128, 128)):
    net = OrderedDict()
    
    net["inp"] = InputLayer(tuple([None, n_chan]) + filter_size)
    
    net["conv1"] = Conv2DLayer(net["inp"], 32, 7, nonlinearity=LeakyRectify(0.2))
    
    net["conv2"] = Conv2DLayer(net["conv1"], 64, 5, stride=2, pad="same")
    net["norm1"] = InstanceNorm(net["conv2"])
    net["lref1"] = NonlinearityLayer(net["norm1"], nonlinearity=LeakyRectify(0.2))
    
    net["conv3"] = Conv2DLayer(net["lref1"], 64, 5, stride=2, pad="same")
    net["norm2"] = InstanceNorm(net["conv3"])
    net["lref2"] = NonlinearityLayer(net["norm2"], nonlinearity=LeakyRectify(0.2))
    
    net["resid1"] = ResidualBlock(net["lref2"], 3)
    for i in range(8):
        net["resid%d"%(i+2)] = ResidualBlock(net["resid%d"%(i+1)], 3)
        
    return net

In [3]:
net = buildGenerator()

In [4]:
out = get_output(net["resid9"])

generate = theano.function([net["inp"].input_var], out)

In [ ]:
import numpy as np

pic = generate(np.random.normal(size=[1, 3, 128, 128]))